## Petroineos Summer Internship 2025 - Data Analysis Coding Challenge
Author: Chun (Johnny) Chan

Goal: complete the PowerPlants class with three functions

In [15]:
# Import necessary libraries
import pandas as pd
import datetime as dt


In [16]:
# Import csv files
database = pd.read_csv("database.csv")
gas_fr_plants = pd.read_csv("gas_fr_plants.csv")
gas_plants = pd.read_csv("gas_plants.csv")
wind_plants = pd.read_csv("wind_plants.csv")

In [17]:
# from power_plants_task.pdf
class PowerPlants(object):
    def __init__(self ):
        self.database_file = 'database.csv'
        self.database = pd.read_csv(self.database_file)
        self.country_map = {'FR': 'France',
                            'GB': 'Great Britain',
                            '0': '0'}
        self.dropped = False

    def load_new_data_from_file(self, file_path: str):
        # load new data to be uploaded
        new_data = pd.read_csv(file_path)

        # # handle missing values as the PDF instructs
        new_data.fillna(0, inplace=True, axis=1)

        # enforce correct data types
        new_data['Volume'] = new_data['Volume'].astype(float)
        new_data['Country '] = new_data['Country '].astype(str)
        new_data['Technology'] = new_data['Technology'].astype(str)
        new_data['SiteName'] = new_data['SiteName'].astype(str)


        # ensure 'Country ' column has no space
        if 'Country ' in new_data.columns:
            new_data.rename(columns={'Country ': 'Country'}, inplace=True)

        # create new full country name column
        new_data['country'] = new_data['Country'].map(self.country_map)

        # enforce correct date format for existing Date column
        new_data['Date'] = pd.to_datetime(new_data['Date'], format='%d/%m/%Y', errors='coerce')

        # create new "date" column in expected format
        new_data['date'] = new_data['Date'].dt.strftime('%Y-%m-%d')

        # create new "updatedby" column
        new_data['updatedby'] = 'petroineos'

        # drop redundant columns
        new_data.drop(['Date', 'Country'], inplace=True, axis=1)

        return new_data


    def save_new_data(self, input_data: pd.DataFrame):
        # concatenate the new data vertically
        self.database = pd.concat([self.database, input_data], ignore_index=True, axis=0, sort=False)

        # getting correct date format for "updatetime" column
        current_datetime = dt.datetime.now()
        formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S.%f')

        # correct updatetime values after concatenation
        self.database['updatetime'] = self.database['updatetime'].fillna(formatted_datetime)

        # Drop redundant columns to match expected structure
        if not self.dropped:
            self.database.drop(['Date', 'Country '], axis=1, inplace=True)
            self.dropped = True

        # Re-ordering the columns into an arbitrary order
        order = ['date', 'country', 'SiteName', 'Technology', 'updatedby', 'updatetime', 'Volume']
        self.database = self.database.loc[:, order]

        # Enforce date time compatibility
        self.database['date'] = pd.to_datetime(self.database['date'])

    def get_data_from_database(self):
        # assuming SiteName is the symbol in question
        result = pp.database[pp.database.groupby('SiteName')['updatetime'].transform('max') == pp.database['updatetime']]
        return result

    def aggregate_data_to_monthly(self):
        # aggregate monthly statistics
        monthly_stats = self.database.groupby(['SiteName', pd.Grouper(key='date', freq='ME')])['Volume'].agg(['mean', 'min', 'max'])

        # only want date as an index
        rotated = monthly_stats.unstack(level='SiteName')

        # flattening the columns
        rotated.columns = [f'{site} {stat}' for stat, site in rotated.columns]
        rotated.reset_index(inplace=True, drop=True)

        return rotated

    def aggregate_data_to_country(self):
        result = self.database.groupby(['country', 'Technology'])['Volume'].sum()
        result.drop('0', inplace=True)
        return result


## Testing our functions

In [18]:
pp = PowerPlants()
new_data = pp.load_new_data_from_file('wind_plants.csv')
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file('gas_plants.csv')
pp.save_new_data(new_data)
new_data = pp.load_new_data_from_file('gas_fr_plants.csv')
pp.save_new_data(new_data)


C:\Users\johnn\AppData\Local\Temp\ipykernel_43640\2543376065.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_data.fillna(0, inplace=True, axis=1)


In [19]:
test_sum = pp.aggregate_data_to_country()
test_sum

country        Technology
France         Gas           7.138749e+06
Great Britain  Gas           2.030437e+07
               Wind          1.426070e+06
Name: Volume, dtype: float64

In [20]:
test_monthly = pp.aggregate_data_to_monthly()
test_monthly

,Blenod-5 mean,Hornsea-1 mean,Hornsea-2 mean,Pembroke-1 mean,Pembroke-2 mean,Blenod-5 min,Hornsea-1 min,Hornsea-2 min,Pembroke-1 min,Pembroke-2 min,Blenod-5 max,Hornsea-1 max,Hornsea-2 max,Pembroke-1 max,Pembroke-2 max
0,5198.806452,502.454543,447.211079,7092.903226,7195.096774,3295.0,44.937929,85.603428,5390.0,5074.0,6890.0,900.648050,892.386165,8905.0,8941.0
1,4889.896552,535.210353,437.902354,6738.620690,6878.551724,3489.0,9.973714,35.733954,5222.0,5051.0,6895.0,980.763621,994.618512,8970.0,8955.0
2,4752.774194,432.654207,592.607677,7499.322581,6605.806452,3122.0,34.240333,122.681263,5196.0,5094.0,6399.0,864.461415,979.909946,8963.0,8685.0
3,4988.466667,561.504570,504.877587,6946.566667,7142.633333,3235.0,67.202179,37.514079,5251.0,5167.0,6970.0,995.133071,999.165096,8941.0,8845.0
4,5012.032258,431.650067,536.694996,7211.290323,6759.806452,3059.0,14.319806,16.694314,5009.0,5039.0,6898.0,913.805516,962.658133,8938.0,8888.0
5,4928.666667,384.724509,387.852090,7003.300000,7368.933333,3083.0,16.042355,1.797161,5001.0,5247.0,6870.0,852.995514,979.952783,8978.0,8934.0
6,4933.677419,471.688183,510.198669,7052.580645,7057.483871,3115.0,8.136290,13.206362,5077.0,5017.0,6680.0,944.258261,850.887204,8983.0,8981.0
7,5296.258065,627.744512,474.907802,7203.032258,7455.225806,3104.0,6.695016,37.231691,5049.0,5262.0,6973.0,988.355244,999.000373,8977.0,8978.0
8,4858.066667,474.785256,466.750343,7382.333333,6596.333333,3110.0,24.260159,80.138479,5149.0,5009.0,6922.0,945.447225,992.225257,8925.0,8931.0
9,4967.740741,524.710537,520.673636,7202.129032,6711.483871,0.0,75.917168,43.367492,5162.0,5101.0,6973.0,990.985647,988.350146,8923.0,8909.0


In [21]:
pp.database

,date,country,SiteName,Technology,updatedby,updatetime,Volume
0,2024-01-01,France,Blenod-5,Gas,petroineos,2025-05-29 10:38:42.401709,6753.0
1,2024-01-02,France,Blenod-5,Gas,petroineos,2025-05-29 10:38:42.401709,3896.0
2,2024-01-03,France,Blenod-5,Gas,petroineos,2025-05-29 10:38:42.401709,3636.0
3,2024-01-04,France,Blenod-5,Gas,petroineos,2025-05-29 10:38:42.401709,5138.0
4,2024-01-05,France,Blenod-5,Gas,petroineos,2025-05-29 10:38:42.401709,5265.0
...,...,...,...,...,...,...,...
7664,NaT,0,0,0,petroineos,2025-05-30 19:36:46.841307,0.0
7665,NaT,0,0,0,petroineos,2025-05-30 19:36:46.841307,0.0
7666,NaT,0,0,0,petroineos,2025-05-30 19:36:46.841307,0.0
7667,NaT,0,0,0,petroineos,2025-05-30 19:36:46.841307,0.0
